In [11]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [26]:
# Offline, 0.5/1 profits, mixed curves, 20 blocks
# Running on c2 on 10/12 2.18
# Change load and see impact on profit

# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_20-04-43"))
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-10_22-08-25"))
rdf["scheduler_metric"] = rdf.apply(lambda row: row.scheduler_metric if row.scheduler == "basic_scheduler" else "Simplex", axis=1)
px.line(
    rdf.sort_values("n_tasks"),
    x="n_tasks",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1_500],
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [28]:
rdf.columns

Index(['scheduler', 'solver', 'scheduler_n', 'scheduler_metric',
       'block_selecting_policy', 'frequency_file', 'n_allocated_tasks',
       'total_tasks', 'realized_profit', 'n_tasks', 'n_initial_blocks',
       'maximum_profit', 'scheduling_time', 'T', 'data_lifetime',
       'mean_task_per_block', 'data_path', 'allocated_tasks_scheduling_delays',
       'metric_recomputation_period', 'normalize_by', 'temperature',
       'time_this_iter_s', 'done', 'timesteps_total', 'episodes_total',
       'training_iteration', 'trial_id', 'experiment_id', 'date', 'timestamp',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'config',
       'time_since_restore', 'timesteps_since_restore',
       'iterations_since_restore'],
      dtype='object')

In [29]:
def map_metric_to_id(row):
    d = {
        "DominantShares": 0,
        "DynamicFlatRelevance": 1,
        "OverflowRelevance":2,
        "SoftKnapsack":3,
        "Simplex":4,
    }
    return d[row]

gnuplot_df = rdf.query("scheduler_metric not in ['Fcfs', 'FlatRelevance']")
gnuplot_df["id"] = gnuplot_df.scheduler_metric.apply(map_metric_to_id)
gnuplot_df = gnuplot_df[["n_tasks", "time_total_s", "id", "scheduler", "scheduler_metric"]].sort_values(["id","time_total_s"])
gnuplot_df.to_csv("/home/pierre/privacypacking-paper/results/offline_mixed_curves/offline_mixed_curves_without_profits_runtime.csv", index=False)

In [22]:
px.line(
    rdf.sort_values("n_tasks"),
    x="n_tasks",
    y="time_total_s",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1_000],
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)